In [ ]:
from time import time

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.keras.utils import to_categorical
import numpy as np

timestamp = int(time())

## Dados

Carregamos os dados já embaralhados divididos em train e test

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Passamos as entradas pra `float` (pra poder manipular), adicionamos a dimensão do canal e pré-tratamos
Como temos um intervalo definido de entrada [0, 255], simplesmente mudamos a escala para [0, 1], o que é bem comum de se fazer em imagens


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train /= 255
x_test /= 255

Passamos os labels pra one-hot encoding (vetor 10-dimensional)

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## Modelo

Camada de entrada (compatível com a forma de x)

In [ ]:
out = entry = Input(shape=x_train.shape[1:])

Camadas de convolução

In [ ]:
out = Conv2D(32, kernel_size=3, strides=1)(out)
out = Activation('relu')(out)
out = MaxPool2D()(out)

Camada de saída com 10 neurônios, cada um responsável por um dígito e aplicação do softmax para obtermos uma distribuição de probabilidade

Antes, transformamos o tensor em um vetor unidimensional. Isso  é necessário para podermos aplicar uma camada densa

In [ ]:
out = Flatten()(out)

out = Dense(10)(out)
out = Activation('softmax')(out)

Definição do modelo em si

In [ ]:
net = Model(entry, out)

Imprimimos a descrição do modelo

In [ ]:
net.summary()

## Treinamento

Definição do custo e da otimização
Custo é a cross-entropia entre saída e resposta
Otimização é a descida de gradientes estocástica

In [ ]:
net.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True),
    metrics=['accuracy'])

Treinamento em si

In [ ]:
net.fit(
    x_train, y_train,
    batch_size=60,
    epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[
        ModelCheckpoint('save/mnist.{epoch:02d}.h5'),
        TensorBoard(log_dir='logs/mnist_{}'.format(timestamp), histogram_freq=1)])

## Inferência

Carrega a imagem, converte para escala de cinza e redimensiona para o tamanho da rede

In [ ]:
image = load_img('data/5.png', grayscale=True, target_size=net.input_shape[1:])

Cria a entrada e infere a saída

In [ ]:
x = np.reshape(image, (1, 28, 28, 1))
y = net.predict(x, verbose=0)[0]

print('É um', np.argmax(y))